In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x241a830
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
plot_path = "plots/angles/mg5_1M"

In [5]:
# ROOT.ildStyle.SetOptStat(1)
ROOT.ildStyle.SetOptStat(0)

In [6]:
%%cpp
using namespace ROOT::VecOps;

In [7]:
df = ROOT.RDataFrame("events", "data/k4gen/mg5/WW250_ll_1M.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/k4gen/mg5/WW250_ll_100k.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/k4gen/whizard/WW250_ll_100k.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/truejet/test/ww_sl/nano.root")
df.Describe()

Dataframe from TChain events in file data/k4gen/mg5/WW250_ll_1M.edm4hep.root

Property                Value
--------                -----
Columns in total           67
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeaders                            ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeaders.eventNumber                ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeaders.runNumber                  ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeaders.timeStamp                  ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeaders.weight                     ROOT::VecOps::RVec<Double_t>                    Dataset
EventHeaders.weights_begin              ROOT::VecOps::RVec<UInt_t>     

In [8]:
# these files are very simple, they have the Ws explicitly and there is only one stable e/p
df = df.Define("e_MC_idx", "ArgMax(MCParticles.PDG == 11 && MCParticles.generatorStatus == 1)")
df = df.Define("enu_MC_idx", "ArgMax(MCParticles.PDG == -12 && MCParticles.generatorStatus == 1)")
df = df.Define("p_MC_idx", "ArgMax(MCParticles.PDG == -11 && MCParticles.generatorStatus == 1)")
df = df.Define("pnu_MC_idx", "ArgMax(MCParticles.PDG == 12 && MCParticles.generatorStatus == 1)")
df = df.Define("Wp_MC_idx", "ArgMax(MCParticles.PDG == 24)")
df = df.Define("Wm_MC_idx", "ArgMax(MCParticles.PDG == -24)")

df = df.Define("e_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[e_MC_idx], MCParticles.momentum.y[e_MC_idx], MCParticles.momentum.z[e_MC_idx], MCParticles.mass[e_MC_idx])")
df = df.Define("enu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[enu_MC_idx], MCParticles.momentum.y[enu_MC_idx], MCParticles.momentum.z[enu_MC_idx], MCParticles.mass[enu_MC_idx])")
df = df.Define("p_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[p_MC_idx], MCParticles.momentum.y[p_MC_idx], MCParticles.momentum.z[p_MC_idx], MCParticles.mass[p_MC_idx])")
df = df.Define("pnu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[pnu_MC_idx], MCParticles.momentum.y[pnu_MC_idx], MCParticles.momentum.z[pnu_MC_idx], MCParticles.mass[pnu_MC_idx])")
df = df.Define("Wp_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wp_MC_idx], MCParticles.momentum.y[Wp_MC_idx], MCParticles.momentum.z[Wp_MC_idx], MCParticles.mass[Wp_MC_idx])")
df = df.Define("Wm_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wm_MC_idx], MCParticles.momentum.y[Wm_MC_idx], MCParticles.momentum.z[Wm_MC_idx], MCParticles.mass[Wm_MC_idx])")

In [9]:

df = df.Define("l_MC_lvec_e", "e_MC_lvec.energy()")
df = df.Define("l_MC_lvec_theta", "e_MC_lvec.theta()")
df = df.Define("l_MC_lvec_cosTheta", "cos(l_MC_lvec_theta)")

df = df.Define("l_MC_lvec_pt", "e_MC_lvec.pt()")
df = df.Define("l_MC_lvec_pz", "e_MC_lvec.pz()")

df = df.Define("e_MC_lvec_e", "e_MC_lvec.energy()")
df = df.Define("enu_MC_lvec_e", "enu_MC_lvec.energy()")
df = df.Define("p_MC_lvec_e", "p_MC_lvec.energy()")
df = df.Define("pnu_MC_lvec_e", "pnu_MC_lvec.energy()")

df = df.Define("e_MC_lvec_px", "e_MC_lvec.px()")
df = df.Define("e_MC_lvec_py", "e_MC_lvec.py()")
df = df.Define("e_MC_lvec_pz", "e_MC_lvec.pz()")

df = df.Define("p_MC_lvec_px", "p_MC_lvec.px()")
df = df.Define("p_MC_lvec_py", "p_MC_lvec.py()")
df = df.Define("p_MC_lvec_pz", "p_MC_lvec.pz()")

df = df.Define("Wm_MC_lvec_px", "Wm_MC_lvec.px()")
df = df.Define("Wm_MC_lvec_py", "Wm_MC_lvec.py()")
df = df.Define("Wm_MC_lvec_pz", "Wm_MC_lvec.pz()")

df = df.Define("Wp_MC_lvec_px", "Wp_MC_lvec.px()")
df = df.Define("Wp_MC_lvec_py", "Wp_MC_lvec.py()")
df = df.Define("Wp_MC_lvec_pz", "Wp_MC_lvec.pz()")

### start of angle calculations
# the simplest is the production angle theta_W (where W stands not for Weinberg in this case but for W-boson)
# following M. Bilenky et al. in Nucl.Phys.B 409 (1993) 22-68 (doi: https://doi.org/10.1016/0550-3213(93)90445-U)
# this is the angle between the W^{-} and the beam e^{-} which in our case is just the Z-axis
# (TODO: check what happens when ddsim adds a crossing angle :/)

df = df.Define("W_minus_lvec", "Wm_MC_lvec")
df = df.Define("W_plus_lvec", "Wp_MC_lvec")
df = df.Define("e_minus_vec", "ROOT::Math::XYZVector(0, 0, 1)")
df = df.Define("e_plus_vec", "ROOT::Math::XYZVector(0, 0, -1)")

df = df.Define("CosTheta_W_minus", "ROOT::Math::VectorUtil::CosTheta(W_minus_lvec, e_minus_vec)")
# FIXME: e_minus_vec?
df = df.Define("CosTheta_W_plus", "ROOT::Math::VectorUtil::CosTheta(W_plus_lvec, e_minus_vec)")

# build custom coordinate system t determine l angles:
# z1 points along the W- flight direction
df = df.Define("z1", "W_minus_lvec.Vect().Unit()")
# y1 is e_minus_vec x W dir/z1
df = df.Define("y1", "e_minus_vec.Cross(z1).Unit()")
# I *think* this is the right order..., no extra .Unit() should be necessary
df = df.Define("x1", "y1.Cross(z1)")
df = df.Define("rot1", "ROOT::Math::Rotation3D(x1, y1, z1).Inverse()")
# the other coordinate system
# df = df.Define("z2", "-W_plus_lvec.Vect().Unit()") # breaks it the same way as when using z1
df = df.Define("z2", "W_plus_lvec.Vect().Unit()")
# yes also e_minus here!
# df = df.Define("y2", "y1")
# df = df.Define("y2", "-y1")
df = df.Define("y2", "e_minus_vec.Cross(z2).Unit()")
# df = df.Define("y2", "e_plus_vec.Cross(z2).Unit()")
# I *think* this is the right order..., no extra .Unit() should be necessary
df = df.Define("x2", "y2.Cross(z2)")
df = df.Define("rot2", "ROOT::Math::Rotation3D(x2, y2, z2).Inverse()")

# get boost
df = df.Define("W_minus_boost", "W_minus_lvec.BoostToCM()")
df = df.Define("W_plus_boost", "W_plus_lvec.BoostToCM()")

df = df.Define("e_boosted_lvec", "ROOT::Math::VectorUtil::boost(e_MC_lvec, W_minus_boost)")
df = df.Define("e_transformed_lvec", "rot1(e_boosted_lvec)")
# df = df.Define("e_transformed_lvec", "rot2(e_boosted_lvec)")
# df = df.Define("e_transformed_lvec", "e_boosted_lvec")
df = df.Define("p_boosted_lvec", "ROOT::Math::VectorUtil::boost(p_MC_lvec, W_plus_boost)")
df = df.Define("p_transformed_lvec", "rot2(p_boosted_lvec)")
# df = df.Define("p_transformed_lvec", "rot1(p_boosted_lvec)")
# df = df.Define("p_transformed_lvec", "p_boosted_lvec")

df = df.Define("e_theta_prime", "e_transformed_lvec.Theta()")
df = df.Define("e_phi_prime", "e_transformed_lvec.Phi()")
df = df.Define("p_theta_prime", "p_transformed_lvec.Theta()")
# df = df.Define("p_phi_prime", "p_transformed_lvec.Phi()")
# df = df.Define("l_theta_prime", "auto theta = l_transformed_lvec.Theta(); return l_MC_charge == 1. ? ROOT::Math::Pi() - theta : theta;")
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); return l_MC_charge == 1. ? phi - ROOT::Math::Pi() : phi;")

# OPAL shift W plus by 180 deg, but this gives it the wrong shape?!
# df = df.Define("l_phi_prime", "auto phi = l_transformed_lvec.Phi(); auto phi_s = l_MC_charge == 1. ? phi + ROOT::Math::Pi() : phi; return phi_s > ROOT::Math::Pi() ? phi_s - 2 * ROOT::Math::Pi() : phi_s;")
df = df.Define("p_phi_prime", "auto phi = p_transformed_lvec.Phi() + ROOT::Math::Pi(); return phi > ROOT::Math::Pi() ? phi - 2 * ROOT::Math::Pi() : phi;")

df = df.Define("e_cosTheta_prime", "cos(e_theta_prime)")
df = df.Define("p_cosTheta_prime", "cos(p_theta_prime)")

In [10]:
# FIXME: for debugging
df = df.Define("weight", "1")
# df = df.Filter("l_MC_lvec_pt > 2", "pt cut")
# df = df.Filter("abs(l_MC_lvec_cosTheta) < 0.975", "cosTheta cut") # ~6.5 deg

In [11]:
h_l_e = df.Histo1D(("", ";E_{e} [GeV]", 300, 0., 150.), "l_MC_lvec_e", "weight")
h_nu_e = df.Histo1D(("", ";E_{#nu} [GeV]", 300, 0., 150.), "enu_MC_lvec_e", "weight")
h_2d_e_e = df.Histo2D(("", ";E_{e-} [GeV];E_{#bar{#nu}} [GeV]", 300, 0., 150., 300, 0., 150.), "e_MC_lvec_e", "enu_MC_lvec_e", "weight")
h_2d_p_e = df.Histo2D(("", ";E_{e+} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "p_MC_lvec_e", "pnu_MC_lvec_e", "weight")
h_2d_lnu_pzpt = df.Histo2D(("", ";P_{z}(e) [GeV];P_{t}(e) [GeV]", 150, -150., 150., 150, 0., 150.), "l_MC_lvec_pz", "l_MC_lvec_pt", "weight")
h_2d_lnu_cosThetapt = df.Histo2D(("", ";cos #theta(e);P_{t}(e) [GeV]", 150, -1., 1., 150, 0., 150.), "l_MC_lvec_cosTheta", "l_MC_lvec_pt", "weight")

h_2d_ep_px = df.Histo2D(("", ";P_{x}(e-) [GeV];P_{x}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_px", "p_MC_lvec_px", "weight")
h_2d_ep_py = df.Histo2D(("", ";P_{y}(e-) [GeV];P_{y}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_py", "p_MC_lvec_py", "weight")
h_2d_ep_pz = df.Histo2D(("", ";P_{z}(e-) [GeV];P_{z}(e+) [GeV]", 600, -150., 150., 600, -150., 150.), "e_MC_lvec_pz", "p_MC_lvec_pz", "weight")

h_2d_WW_px = df.Histo2D(("", ";P_{x}(W-) [GeV];P_{x}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_px", "Wp_MC_lvec_px", "weight")
h_2d_WW_py = df.Histo2D(("", ";P_{y}(W-) [GeV];P_{y}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_py", "Wp_MC_lvec_py", "weight")
h_2d_WW_pz = df.Histo2D(("", ";P_{z}(W-) [GeV];P_{z}(W+) [GeV]", 600, -150., 150., 600, -150., 150.), "Wm_MC_lvec_pz", "Wp_MC_lvec_pz", "weight")

h_e_px = df.Histo1D(("", ";P_{x}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_px", "weight")
h_e_py = df.Histo1D(("", ";P_{y}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_py", "weight")
h_e_pz = df.Histo1D(("", ";P_{z}(e-) [GeV]", 600, -150., 150.), "e_MC_lvec_pz", "weight")

h_p_px = df.Histo1D(("", ";P_{x}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_px", "weight")
h_p_py = df.Histo1D(("", ";P_{y}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_py", "weight")
h_p_pz = df.Histo1D(("", ";P_{z}(e+) [GeV]", 600, -150., 150.), "p_MC_lvec_pz", "weight")

h_CosTheta_W_minus = df.Histo1D(("", ";cos #theta_{W^{-}}", 40, -1., 1.), "CosTheta_W_minus", "weight")
h_CosTheta_W_plus = df.Histo1D(("", ";cos #theta_{W^{+}}", 40, -1., 1.), "CosTheta_W_plus", "weight")
# h_l_theta_prime = df.Histo1D("l_theta_prime", "weight")
h_l_plus_theta_prime = df.Histo1D("p_theta_prime", "weight")
h_l_plus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{+}}", 40, -1., 1.), "p_cosTheta_prime", "weight")
h_l_plus_phi_prime = df.Histo1D(("", ";#phi*_{e+}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "p_phi_prime", "weight")
h_l_minus_theta_prime = df.Histo1D("e_theta_prime", "weight")
h_l_minus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{+}}", 40, -1., 1.), "e_cosTheta_prime", "weight")
h_l_minus_phi_prime = df.Histo1D(("", ";#phi*_{e-}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "e_phi_prime", "weight")

h_l_cosTheta_prime = h_l_plus_cosTheta_prime.Clone()
h_l_cosTheta_prime.Add(h_l_minus_cosTheta_prime.GetPtr())
h_l_cosTheta_prime.SetTitle(";cos #theta*_{e^{#pm}}")
h_l_phi_prime = h_l_plus_phi_prime.Clone()
h_l_phi_prime.Add(h_l_minus_phi_prime.GetPtr())
h_l_phi_prime.SetTitle(";#phi*_{e#pm}")

In [12]:
h_l_phi_prime.SetMinimum(0)
h_l_plus_phi_prime.SetMinimum(0)
h_l_minus_phi_prime.SetMinimum(0)

In [13]:
# c_l_e = ROOT.TCanvas()
# h_l_e.Draw()
# c_l_e.Draw()

# c_nu_e = ROOT.TCanvas()
# h_nu_e.Draw()
# c_nu_e.Draw()

# c_2d_ep_px = ROOT.TCanvas()
# h_2d_ep_px.Draw("colz0")
# c_2d_ep_px.Draw()

# c_e_px = ROOT.TCanvas()
# h_e_px.Draw("hist")
# h_p_px.SetLineColor(ROOT.kRed)
# h_p_px.Draw("same hist")
# c_e_px.Draw()

# c_2d_ep_py = ROOT.TCanvas()
# h_2d_ep_py.Draw("colz0")
# c_2d_ep_py.Draw()

# c_e_py = ROOT.TCanvas()
# h_e_py.Draw("hist")
# h_p_py.SetLineColor(ROOT.kRed)
# h_p_py.Draw("same hist")
# c_e_py.Draw()

# c_2d_ep_pz = ROOT.TCanvas()
# h_2d_ep_pz.Draw("colz0")
# c_2d_ep_pz.Draw()

# c_e_pz = ROOT.TCanvas()
# h_e_pz.Draw("hist")
# h_p_pz.SetLineColor(ROOT.kRed)
# h_p_pz.Draw("same hist")
# c_e_pz.Draw()

# c_2d_WW_px = ROOT.TCanvas()
# h_2d_WW_px.Draw("colz0")
# c_2d_WW_px.Draw()

# c_2d_WW_py = ROOT.TCanvas()
# h_2d_WW_py.Draw("colz0")
# c_2d_WW_py.Draw()

# c_2d_WW_pz = ROOT.TCanvas()
# h_2d_WW_pz.Draw("colz0")
# c_2d_WW_pz.Draw()

# c_2d_e_e = ROOT.TCanvas()
# h_2d_e_e.Draw("colz0")
# c_2d_e_e.Draw()

# c_2d_p_e = ROOT.TCanvas()
# h_2d_p_e.Draw("colz0")
# c_2d_p_e.Draw()

# c_2d_lnu_pzpt = ROOT.TCanvas()
# h_2d_lnu_pzpt.Draw("colz0")
# c_2d_lnu_pzpt.Draw()

# c_2d_lnu_cosThetapt = ROOT.TCanvas()
# h_2d_lnu_cosThetapt.Draw("colz0")
# c_2d_lnu_cosThetapt.Draw()

c_CosTheta_W_minus = ROOT.TCanvas()
h_CosTheta_W_minus.Draw()
# stats = h_CosTheta_W_minus.FindObject("stats")
# stats.SetX1NDC(0.2)
# stats.SetX2NDC(0.5)
# stats.SetY1NDC(0.5)
# stats.SetY2NDC(0.8)
c_CosTheta_W_minus.Draw()
c_CosTheta_W_minus.SaveAs(f"{plot_path}/cosThetaWm.pdf")

# c_CosTheta_W_plus = ROOT.TCanvas()
# h_CosTheta_W_plus.Draw()
# c_CosTheta_W_plus.Draw()

# c_l_theta_prime = ROOT.TCanvas()
# h_l_theta_prime.Draw()
# c_l_theta_prime.Draw()

c_l_cosTheta_prime = ROOT.TCanvas()
h_l_cosTheta_prime.Draw()
c_l_cosTheta_prime.Draw()
c_l_cosTheta_prime.SaveAs(f"{plot_path}/cosThetaPrime_l.pdf")

c_l_phi_prime = ROOT.TCanvas()
h_l_phi_prime.Draw()
c_l_phi_prime.Draw()
c_l_phi_prime.SaveAs(f"{plot_path}/PhiPrime_l.pdf")

# c_l_plus_theta_prime = ROOT.TCanvas()
# h_l_plus_theta_prime.Draw()
# c_l_plus_theta_prime.Draw()

c_l_plus_cosTheta_prime = ROOT.TCanvas()
h_l_plus_cosTheta_prime.Draw()
c_l_plus_cosTheta_prime.Draw()

c_l_plus_phi_prime = ROOT.TCanvas()
h_l_plus_phi_prime.Draw()
c_l_plus_phi_prime.Draw()

# c_l_minus_theta_prime = ROOT.TCanvas()
# h_l_minus_theta_prime.Draw()
# c_l_minus_theta_prime.Draw()

c_l_minus_cosTheta_prime = ROOT.TCanvas()
h_l_minus_cosTheta_prime.Draw()
c_l_minus_cosTheta_prime.Draw()

c_l_minus_phi_prime = ROOT.TCanvas()
h_l_minus_phi_prime.SetMinimum(0)
h_l_minus_phi_prime.Draw()
c_l_minus_phi_prime.Draw()

# c_CosTheta_W_minus_LT = ROOT.TCanvas()
# h_CosTheta_W_minus_LT.Draw()
# c_CosTheta_W_minus_LT.Draw()

Info in <TCanvas::Print>: pdf file plots/angles/mg5_1M/cosThetaWm.pdf has been created
Info in <TCanvas::Print>: pdf file plots/angles/mg5_1M/cosThetaPrime_l.pdf has been created
Info in <TCanvas::Print>: pdf file plots/angles/mg5_1M/PhiPrime_l.pdf has been created
